In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

  Using cached openmeteo_requests-1.2.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached openmeteo_sdk-1.10.0-py3-none-any.whl.metadata (13 kB)
  Using cached flatbuffers-24.3.6-py2.py3-none-any.whl.metadata (849 bytes)
Using cached openmeteo_requests-1.2.0-py3-none-any.whl (5.5 kB)
Using cached openmeteo_sdk-1.10.0-py3-none-any.whl (12 kB)
Using cached flatbuffers-24.3.6-py2.py3-none-any.whl (26 kB)
  Using cached requests_cache-1.2.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached retry_requests-2.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached cattrs-23.2.3-py3-none-any.whl.metadata (10 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl.metadata (3.1 kB)
Using cached requests_cache-1.2.0-py3-none-any.whl (61 kB)
Using cached retry_requests-2.0.0-py3-none-any.whl (15 kB)
Using cached cattrs-23.2.3-py3-none-any.whl (57 kB)
Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)


In [3]:
import openmeteo_requests
from openmeteo_sdk.Variable import Variable
from openmeteo_sdk.Aggregation import Aggregation
import requests_cache
import pandas as pd
from retry_requests import retry

In [5]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://ensemble-api.open-meteo.com/v1/ensemble"
params = {
	"latitude": 51.51,
	"longitude": 0.1276,
	"hourly": "direct_radiation_instant",
	"past_days": 5,
	"forecast_days": 1,
	"models": "icon_seamless"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data
hourly = response.Hourly()
hourly_variables = list(map(lambda i: hourly.Variables(i), range(0, hourly.VariablesLength())))
hourly_direct_radiation_instant = filter(lambda x: x.Variable() == Variable.direct_radiation_instant, hourly_variables)

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
# Process all members
for variable in hourly_direct_radiation_instant:
	member = variable.EnsembleMember()
	hourly_data[f"direct_radiation_instant_member{member}"] = variable.ValuesAsNumpy()

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.head()


Coordinates 51.5°N 0.11999988555908203°E
Elevation 4.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,direct_radiation_instant_member0,direct_radiation_instant_member1,direct_radiation_instant_member2,direct_radiation_instant_member3,direct_radiation_instant_member4,direct_radiation_instant_member5,direct_radiation_instant_member6,direct_radiation_instant_member7,direct_radiation_instant_member8,...,direct_radiation_instant_member30,direct_radiation_instant_member31,direct_radiation_instant_member32,direct_radiation_instant_member33,direct_radiation_instant_member34,direct_radiation_instant_member35,direct_radiation_instant_member36,direct_radiation_instant_member37,direct_radiation_instant_member38,direct_radiation_instant_member39
0,2024-03-02 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-03-02 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-03-02 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-03-02 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-03-02 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
